In [1]:
!pip install -U huggingface_hub

   ---------------------------------------- 0.0/402.6 kB ? eta -:--:--
   -------- ------------------------------- 81.9/402.6 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 402.6/402.6 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-huggingface 0.1.4 requires huggingface-hub<0.21.0,>=0.20.3, but you have huggingface-hub 0.23.4 which is incompatible.


In [2]:
from huggingface_hub import login
login()

In [2]:
!huggingface-cli download --resume-download openai/clip-vit-large-patch14-336 --local-dir openai/clip-vit-large-patch14-336 --local-dir-use-symlinks False

D:\Llava\openai\clip-vit-large-patch14-336


D:\Anaconda\Lib\site-packages\huggingface_hub\commands\download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
D:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]Downloading 'special_tokens_map.json' to 'openai\clip-vit-large-patch14-336\.huggingface\download\special_tokens_map.json.9bfb42aa97dcd61e89f279ccaee988bccb4fabae.incomplete'
Download complete. Moving file to openai\clip-vit-large-patch14-336\special_tokens_map.json
Download complete. Moving file to openai\clip-vit-large-patch14-336\README.md
Download complete. Moving file to openai\clip-vit-large-patch14-336\config.json
Download complete. Moving file to openai

In [ ]:
!huggingface-cli download --resume-download google/gemma-1.1-2b-it --local-dir google/gemma-1.1-2b-it --local-dir-use-symlinks False

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoProcessor
import torch

In [15]:
clip_model_name_or_path = r"D:\Llava\openai\clip-vit-large-patch14-336"
gemma_model_name_or_path = r"D:\Llava\meta-llama\Meta-Llama-3-8B-Instruct"

In [3]:
clip_model = AutoModel.from_pretrained(clip_model_name_or_path)
llm_model = AutoModelForCausalLM.from_pretrained(gemma_model_name_or_path,load_in_4bit = True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
d:\Anaconda\Lib\site-packages\transformers\quantizers\auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [4]:
llm_tokenizer = AutoTokenizer.from_pretrained(gemma_model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
llm_tokenizer.encode("<image>")

[128000, 128002]

In [6]:
from transformers import LlavaForConditionalGeneration, LlavaConfig

In [7]:
vision_config = clip_model.vision_model.config
text_config = llm_model.config
configuration = LlavaConfig(vision_config, text_config)
model = LlavaForConditionalGeneration(configuration)

In [8]:
model.config.pad_token_id = llm_tokenizer.pad_token_id

In [9]:
model.config.bos_token_id = llm_tokenizer.bos_token_id
model.config.eos_token_id = llm_tokenizer.eos_token_id

In [13]:
model.config.image_token_index = llm_tokenizer.encode("<image>")[0]
model.config.image_token_index

128000

In [14]:
model.save_pretrained(r"llava_model\model002",)

In [22]:
llm_tokenizer.save_pretrained(r"llava_model\model002")

('llava_model\\model002\\tokenizer_config.json',
 'llava_model\\model002\\special_tokens_map.json',
 'llava_model\\model002\\tokenizer.json')

In [21]:
autoprocessor = AutoProcessor.from_pretrained(clip_model_name_or_path)
autoprocessor.save_pretrained(r"llava_model\temp")

[]

In [1]:
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, LlavaProcessor, GemmaTokenizer, CLIPImageProcessor

class MyLlavaProcessor(LlavaProcessor):
    tokenizer_class = ("LlamaTokenizer", "LlamaTokenizerFast", "GemmaTokenizer")

# model_id = "test_model_copy/model001"
model_id = r"llava_model\model001"  #


model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(0)

processor = MyLlavaProcessor(image_processor=CLIPImageProcessor.from_pretrained(model_id), tokenizer=GemmaTokenizer.from_pretrained(model_id))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model.push_to_hub("init_gemma_clip_llava")

Removed shared tensor {'language_model.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/642M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/princepride/init_gemma_clip_llava/commit/acaf708b16571cc39cc51d6fedd256078b370923', commit_message='Upload LlavaForConditionalGeneration', commit_description='', oid='acaf708b16571cc39cc51d6fedd256078b370923', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, LlavaProcessor, GemmaTokenizer, CLIPImageProcessor

class MyLlavaProcessor(LlavaProcessor):
    tokenizer_class = ("LlamaTokenizer", "LlamaTokenizerFast", "GemmaTokenizer")

# model_id = "test_model_copy/model001"
model_id = r"llava_model\model001"  #


model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(0)

processor = MyLlavaProcessor(image_processor=CLIPImageProcessor.from_pretrained(model_id), tokenizer=GemmaTokenizer.from_pretrained(model_id))
processor.tokenizer.add_bos_token = True
prompt_text = r"<image>\nWhat are these?"
image_file = r"000000039769.jpg"
raw_image = Image.open(image_file)
inputs = processor(prompt_text, raw_image, return_tensors="pt").to(0, torch.float16)


output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0], skip_special_tokens=False))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

d:\Anaconda\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos><image>\nWhat are these?????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????


In [12]:
inputs

{'input_ids': tensor([[     2,      7, 235286, 235254,   1841,    708,   1450, 235336]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[[[ 0.5435,  0.6455,  0.5581,  ...,  0.0909,  0.0033, -0.0696],
          [ 0.5435,  0.6167,  0.5435,  ...,  0.1201,  0.0179,  0.0617],
          [ 0.5581,  0.5581,  0.6602,  ...,  0.0909,  0.0764,  0.0617],
          ...,
          [ 1.8281,  1.8867,  1.8281,  ...,  1.4053,  1.4482,  1.5654],
          [ 1.8574,  1.9014,  1.8721,  ...,  1.4775,  1.4053,  1.4922],
          [ 1.8721,  1.9014,  1.9014,  ...,  1.4053,  1.2148,  1.4775]],

         [[-1.3623, -1.2715, -1.3770,  ..., -1.4219, -1.4824, -1.5117],
          [-1.3320, -1.2422, -1.3467,  ..., -1.4219, -1.4824, -1.4219],
          [-1.2422, -1.2871, -1.1973,  ..., -1.4668, -1.4668, -1.4824],
          ...,
          [ 0.0789,  0.1239,  0.0338,  ..., -0.7168, -0.6567, -0.5664],
          [ 0.1089,  0.1089,  0.0789,  ..., -0.

In [2]:
processor.tokenizer.add_bos_token

False

In [4]:
processor.tokenizer.add_eos_token

False

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(r"D:\Llava\google\gemma-1.1-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    r"D:\Llava\google\gemma-1.1-2b-it",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model.to("cuda:0")

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [7]:
input_text = ["Write me a poem about Machine Learning.", "Help me calculate what's the answer of 1 + 1?"]
input_ids = tokenizer(input_text, return_tensors="pt", padding=True)
input_ids.to("cuda:0")
outputs = model.generate(**input_ids, max_length=512)
print(tokenizer.decode(outputs[0]))
print(tokenizer.decode(outputs[1]))

d:\Anaconda\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<pad><pad><pad><pad><pad><pad><pad><bos>Write me a poem about Machine Learning.

In circuits of logic, a mind unseen,
A tapestry of data, a world serene.
Algorithms dance, a symphony of code,
Learning from experience, a journey bold.

From images and texts, to numbers and dates,
The machine learns, a wondrous chase.
Pattern recognition, a task of the past,
Now automated, a future unsurpassed.

In predictive modeling, forecasts so bold,
Predicting future trends, stories untold.
Clustering algorithms, sorting and grouping,
Insights hidden, where knowledge is winning.

From medical diagnosis to financial trends,
Machine learning's impact, a guiding hand.
It revolutionizes, a transformative force,
Unlocking possibilities, forevermore.

So let us embrace this technological might,
Harnessing its power, shining bright.
For in the realm of machines, a new dawn breaks,
Where human and artificial, hand in graceful wakes.<eos>
<bos>Help me calculate what's the answer of 1 + 1?

The answer is 2.



In [8]:
input_ids

{'input_ids': tensor([[     0,      0,      0,      0,      0,      0,      0,      2,   5559,
            682,    476,  19592,   1105,  13403,  14715, 235265],
        [     2,  14795,    682,  13988,   1212, 235303, 235256,    573,   3448,
            576, 235248, 235274,    963, 235248, 235274, 235336]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
print(tokenizer.decode(input_ids.input_ids[0]))

<pad><pad><pad><pad><pad><pad><pad><bos>Write me a poem about Machine Learning.


In [8]:
input_ids

{'input_ids': tensor([[     2,   5559,    682,    476,  19592,   1105,  13403,  14715, 235265]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
print(tokenizer.decode(outputs[0],skip_special_tokens=True)[len(input_text):])



In circuits of logic, a mind unseen,

